In [1]:
# Importing Libraries
import os
from os.path import exists
import numpy as np
import pickle as pkl
from tqdm.notebook import tqdm_notebook as bar
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [2]:
# Importing Data
print('Checking for Binary Data Files...')
if exists(r'./bin/feature_train.pkl'):
    print('Data Files Found')
    print('Loading Files...')
    X_train = pkl.load(open(r'./bin/feature_train.pkl', 'rb'))
    y_train = pkl.load(open(r'./bin/class_train.pkl', 'rb'))
    X_test = pkl.load(open(r'./bin/feature_test.pkl', 'rb'))
    y_test = pkl.load(open(r'./bin/class_test.pkl', 'rb'))
    print('File Loading Completed Successfully')
else:
    print('Data Files Not Found')
    print('Run data_preparation.py File first and then Re-Run this Cell')

Checking for Binary Data Files...
Data Files Found
Loading Files...
File Loading Completed Successfully


In [3]:
# Data Size
print(f'Training: {X_train.shape, y_train.shape}\nTesting: {X_test.shape, y_test.shape}')

Training: ((5000, 128, 128, 3), (5000,))
Testing: ((1000, 128, 128, 3), (1000,))


In [4]:
# Convolution Neural Network
model = Sequential()

# First Convolution and Max Pooling Layer
conv_layer_1 = Conv2D(64, (3, 3), activation='relu')
max_pool_layer_1 = MaxPooling2D((2, 2))

# Second Convolution and Max Pooling Layer
conv_layer_2 = Conv2D(64, (3, 3), activation='relu')
max_pool_layer_2 = MaxPooling2D((2, 2))

# Third Convolution and Max Pooling Layer
conv_layer_3 = Conv2D(64, (3, 3), activation='relu')
max_pool_layer_3 = MaxPooling2D((2, 2))

# Flattening Layer
flatten_layer = Flatten()

# Hidden Layers
hidden_layer_1 = Dense(128, input_shape=X_train[0].shape, activation='relu')
hidden_layer_2 = Dense(64, activation='relu')

# Output Layer
output_layer = Dense(2, activation='softmax')

# Adding Layers to Neural Network
model.add(conv_layer_1)
model.add(max_pool_layer_1)
model.add(conv_layer_2)
model.add(max_pool_layer_2)
model.add(conv_layer_3)
model.add(max_pool_layer_3)
model.add(flatten_layer)
model.add(hidden_layer_1)
model.add(hidden_layer_2)
model.add(output_layer)

In [5]:
# Compiling Model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
# Setting Up Callback
stopper = EarlyStopping(
    monitor='accuracy', 
    min_delta=0, patience=3, 
    verbose=0, 
    mode='auto', 
    baseline=None, 
    restore_best_weights=True
)

In [7]:
# Fitting Data
history = model.fit(X_train, y_train, epochs=10, callbacks=[stopper], validation_split=0.1)

Epoch 1/10
141/141 [==============================] - 126s 884ms/step - loss: 0.6923 - accuracy: 0.5300 - val_loss: 0.6836 - val_accuracy: 0.6020
Epoch 2/10
141/141 [==============================] - 128s 911ms/step - loss: 0.6556 - accuracy: 0.6204 - val_loss: 0.6419 - val_accuracy: 0.6520
Epoch 3/10
141/141 [==============================] - 127s 901ms/step - loss: 0.5981 - accuracy: 0.6744 - val_loss: 0.5924 - val_accuracy: 0.7240
Epoch 4/10
141/141 [==============================] - 128s 905ms/step - loss: 0.5371 - accuracy: 0.7216 - val_loss: 0.5787 - val_accuracy: 0.7220
Epoch 5/10
141/141 [==============================] - 127s 897ms/step - loss: 0.4614 - accuracy: 0.7802 - val_loss: 0.5608 - val_accuracy: 0.7340
Epoch 6/10
141/141 [==============================] - 128s 906ms/step - loss: 0.3303 - accuracy: 0.8502 - val_loss: 0.6336 - val_accuracy: 0.7220
Epoch 7/10
141/141 [==============================] - 126s 897ms/step - loss: 0.2052 - accuracy: 0.9207 - val_loss: 0.8032 -

In [8]:
# Summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 61, 61, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12544)             0

In [9]:
# Best Results
print('Best Results')
for cat in history.history.keys():
    print(f'{cat}: {history.history[cat][-1]}')

Best Results
loss: 0.06782330572605133
accuracy: 0.9764444231987
val_loss: 1.2070155143737793
val_accuracy: 0.7200000286102295


In [10]:
# Evaluation
model.evaluate(X_test, y_test)

32/32 [==============================] - 7s 225ms/step - loss: 1.4964 - accuracy: 0.6810


[1.4964436292648315, 0.6809999942779541]